## Installation on Google Colab

The transformers package is not installed by default on Google Colab. So let's install it with pip: 

In [1]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.5MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 31.9MB/s 
     |████████████████████████████████| 901kB 34.5MB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 


## Sentiment analysis in english

In this article, we will use the high-level pipeline interface, which makes it extremely easy to use pre-trained transformer models.

Basically, we just need to tell the pipeline what we want to do, and possibly to tell it which model to use for this task.

Here we're going to do sentiment analysis in English, so we select the `sentiment-analysis` task, and the default model: 

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

The pipeline is ready, and we can now use it: 

In [3]:
classifier(["this is a great tutorial, thank you", 
            "your content just sucks"])

[{'label': 'POSITIVE', 'score': 0.9998582601547241},
 {'label': 'NEGATIVE', 'score': 0.9971919059753418}]

We sent two sentences through the pipeline. The first one is predicted to be positive and the second one negative with very high confidence. 

Sounds good! 

Now let's see what happens if we send in french sentences: 

In [4]:
classifier(["Ton tuto est vraiment bien", 
            "il est complètement nul"])

[{'label': 'POSITIVE', 'score': 0.7650704979896545},
 {'label': 'POSITIVE', 'score': 0.8282670974731445}]

This time, the classification does not work... 

Indeed, the second sentence, which means "this tutorial is complete crap", is classified as positive. 

That's not a surprise: the default model for the sentiment analysis task has been trained on English text, so it does not understand French.

### Sentiment analysis in Dutch, German, French, Spanish and Italian

So what can you do if you want to work with text in another language, say French? 

You just need to search the hub for a [french classification model](https://huggingface.co/models?filter=fr&pipeline_tag=text-classification&sort=downloads). 

Several models are available, and I decided to select [nlptown/bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment). 

We can specify this model as the one to be used when we create our `sentiment-analysis` pipeline: 

In [5]:
multilang_classifier = pipeline("sentiment-analysis", 
                                model="nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
multilang_classifier(["Ton tuto est vraiment bien", 
                      "il est complètement nul"])

[{'label': '5 stars', 'score': 0.5787978172302246},
 {'label': '1 star', 'score': 0.9223358035087585}]

And it worked! The second sentence is properly classified as very negative. 

You might be wondering while the confidence for the first sentence is lower. I'm pretty sure that it's because this sentence scores high on '4 stars' as well. 

Now let's try with an actual review for a restaurant near my place: 

In [8]:
import pprint
sentence="Contente de pouvoir retourner au restaurant... Quelle déception... L accueil peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me proposer à la place... Une pizza pas assez cuite et pour finir une glace pleine de glaçons... Et au gout très fade... Je pensais que les serveuses seraient plus aimable à l idée de retrouver leur clientèle.. Dommage"
pprint.pprint(sentence)

('Contente de pouvoir retourner au restaurant... Quelle déception... L accueil '
 'peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me '
 'proposer à la place... Une pizza pas assez cuite et pour finir une glace '
 'pleine de glaçons... Et au gout très fade... Je pensais que les serveuses '
 'seraient plus aimable à l idée de retrouver leur clientèle.. Dommage')


In [9]:
multilang_classifier([sentence])

[{'label': '2 stars', 'score': 0.5843755602836609}]

2 stars! on Google Review, this review has 1 star. Not a bad prediction. 

### Translation 

In [ ]:
en_to_fr = pipeline("translation_en_to_fr", 
                    model="Helsinki-NLP/opus-mt-en-fr")

In [ ]:
en_to_fr("your tutorial is really good")

[{'translation_text': 'votre tutoriel est vraiment bon'}]

In [ ]:
fr_to_en = pipeline("translation_fr_to_en", 
                    model="Helsinki-NLP/opus-mt-fr-en")

In [ ]:
fr_to_en("ton tutoriel est super")

[{'translation_text': 'Your tutorial is great.'}]

### Zero-shot classification in french

In [ ]:
classifier = pipeline("zero-shot-classification", 
                      model="BaptisteDoyen/camembert-base-xlni")

In [ ]:
sequence = "Colin est en train d'écrire un article au sujet du traitement du langage naturel"
candidate_labels = ["science","politique","education", "news"]
classifier(sequence, candidate_labels)     

{'labels': ['science', 'news', 'education', 'politique'],
 'scores': [0.4613836407661438,
  0.20861364901065826,
  0.20573210716247559,
  0.12427058815956116],
 'sequence': "Colin est en train d'écrire un article au sujet du traitement du langage naturel"}

### Writing summaries in French

In [ ]:
summarizer = pipeline("summarization", 
                       model="moussaKam/barthez-orangesum-title")

In [ ]:
import pprint
sentence = "Le premier tour des élections régionales, dimanche 20 juin, a été marqué par un niveau d’abstention inédit (66,7 %). Une première clé est de savoir si ce second tour mobilisera davantage les électeurs que le premier. Il a également fait apparaître un paysage politique fragmenté. Ce morcellement se retrouvera lors du second tour, dimanche 27 juin, lui aussi inédit à bien des égards."
pprint.pprint(sentence)

('Le premier tour des élections régionales, dimanche 20 juin, a été marqué par '
 'un niveau d’abstention inédit (66,7 %). Une première clé est de savoir si ce '
 'second tour mobilisera davantage les électeurs que le premier. Il a '
 'également fait apparaître un paysage politique fragmenté. Ce morcellement se '
 'retrouvera lors du second tour, dimanche 27 juin, lui aussi inédit à bien '
 'des égards.')


In [ ]:
summarizer(sentence, max_length=50)

[{'summary_text': "Régionales : le niveau d'abstention inédit au second tour"}]

### Named entity recognition

In [ ]:
ner = pipeline("token-classification", model="Jean-Baptiste/camembert-ner")

In [ ]:
nes = ner("Colin est parti à Saint-André acheter de la mozzarella")
pprint.pprint(nes)

[{'end': 5,
  'entity': 'PER',
  'index': 1,
  'score': 0.94243556,
  'start': 0,
  'word': '▁Colin'},
 {'end': 23,
  'entity': 'LOC',
  'index': 5,
  'score': 0.99605554,
  'start': 17,
  'word': '▁Saint'},
 {'end': 24,
  'entity': 'LOC',
  'index': 6,
  'score': 0.9967083,
  'start': 23,
  'word': '-'},
 {'end': 29,
  'entity': 'LOC',
  'index': 7,
  'score': 0.99609375,
  'start': 24,
  'word': 'André'}]


In [ ]:
cur = None
agg = []
for ne in nes: 
  entity=ne['entity']
  if entity != cur: 
    if not cur: 
      cur=entity
    if agg: 
      print(cur, ner.tokenizer.convert_tokens_to_string(agg))
      agg = []
      cur = entity
  agg.append(ne['word'])
print(cur, ner.tokenizer.convert_tokens_to_string(agg))


PER Colin
LOC Saint-André
